<a href="https://colab.research.google.com/github/El-Nico/tensorflow-notes/blob/main/possibly_necessary_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#learning rate

In [1]:
#creating a learning rate scheduler callback
#lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch/20))# traverse a set of learning rates strating from 1e-4, increasing by 10**(epoch/20) every epoch

In [2]:
# Checkout the history
#pd.DataFrame(history.history).plot(figsize=(10,7), xlabel="epochs");

In [ ]:
# Plot the learning rate versus the loss
# lrs = 1e-4 * (10 ** (np.arange(100)/20))
# plt.figure(figsize=(10, 7))
# plt.semilogx(lrs, history.history["loss"]) # we want the x-axis (learning rate) to be log scale
# plt.xlabel("Learning Rate")
# plt.ylabel("Loss")
# plt.title("Learning rate vs. loss");

![how to select the best learning rate](https://raw.githubusercontent.com/El-Nico/tensorflow-notes/main/tensorflow_notes_images/learning_rate.png)